In [4]:
import pandas as pd
import numpy as np
import ldtk

In [5]:
target_path = '/Users/u5500483/Documents/GitHub/Paper_II/'
filter_path = '/Users/u5500483/Documents/GitHub/Paper_II/filters_limb_dark/'

In [7]:
def create_ngts():
    df = pd.read_csv(filter_path + 'NGTS_transmission_and_atmosphere_MGunther.txt', comment='#', header=None, names=['wavelength','response'])
    return ldtk.TabulatedFilter('NGTS', df.wavelength.values, df.response.values)

def create_tess():
    df = pd.read_table(filter_path + 'TESS.dat', header=None, names=['wavelength','response'], sep='    ')
    df['wavelength'] = df['wavelength']*0.1
    return ldtk.TabulatedFilter('TESS', df.wavelength.values, df.response.values)

def create_SLOAN_g():
    df = pd.read_table(filter_path + 'SLOAN_SDSS.gprime_filter.dat', header=None, names=['wavelength','response'])
    df['wavelength'] = df['wavelength'].astype('float')*0.1
    return ldtk.TabulatedFilter('SLOAN-g', df.wavelength.values, df.response.values)


def create_L():
    df = pd.read_table(filter_path + 'L.dat', header=None, names=['wavelength','response'])
    df['wavelength'] = df['wavelength'].astype('float')*0.1
    return ldtk.TabulatedFilter('L', df.wavelength.values, df.response.values)


# def create_wasp():
#     df = pd.read_table('/Users/u5500483/Downloads/filters/WASP.dat', header=None, names=['wavelength','response'], delim_whitespace=True)
#     df['wavelength'] = df['wavelength'].astype('float')*0.1
#     return ldtk.TabulatedFilter('WASP', df.wavelength.values, df.response.values)
#
# def create_Johnson(filter):
#     df_path = '/Users/u5500483/Downloads/filters/Generic_Johnson.{}.dat'.format(filter)
#     df = pd.read_table(df_path, header=None, names=['wavelength','response'], sep=' ')
#     df['wavelength'] = df['wavelength']*0.1
#     return ldtk.TabulatedFilter(filter, df.wavelength.values, df.response.values)


In [8]:
ngts = create_ngts()
L = create_L()
# tess = create_tess()

# Limb darkening Coefficients

In [11]:
stellar_df = pd.read_csv(target_path+'stellar_params.dat', comment='#', names=['Parameter','median','upper','lower','3sig_low','3sig_up'], delim_whitespace=True)
display(stellar_df)

,Parameter,median,upper,lower,3sig_low,3sig_up
0,teff,6410.00,125.000,125.000,NaN,NaN
1,logg,4.10,0.012,0.012,NaN,NaN
2,[Fe/H],0.16,0.100,0.100,NaN,NaN


In [12]:
filter_names = np.array(['NGTS', 'L'])
filters = np.array([ngts, L])
#Enter stellar parameters in form (median, 1-sig error). I pull these values from ARIADNE fits but catalogue values are fine
sc = ldtk.LDPSetCreator(teff=(stellar_df.loc[stellar_df.Parameter=='teff', 'median'].item(),np.max([stellar_df.loc[stellar_df.Parameter=='teff', 'upper'].item(), stellar_df.loc[stellar_df.Parameter=='teff', 'lower'].item()])),    # First col value, second col plusminus error
                  logg=(stellar_df.loc[stellar_df.Parameter=='logg', 'median'].item(),np.max([stellar_df.loc[stellar_df.Parameter=='logg', 'upper'].item(), stellar_df.loc[stellar_df.Parameter=='logg', 'lower'].item()])),
                      z=(stellar_df.loc[stellar_df.Parameter=='[Fe/H]', 'median'].item(),np.max([stellar_df.loc[stellar_df.Parameter=='[Fe/H]', 'upper'].item(), stellar_df.loc[stellar_df.Parameter=='[Fe/H]', 'lower'].item()])),
                     filters=filters)
ps = sc.create_profiles(nsamples=2000)                # Create the limb darkening profiles
ps.resample_linear_z(300)
cq,eq = ps.coeffs_qd(do_mc=True)         # Estimate quadratic law coefficients
for i, filter in enumerate(filter_names):
    print(filter)
    print('c1/c2 values:')
    print(cq[i,0],eq[i,0])
    print(cq[i,1],eq[i,1])
print('....')
for i, filter in enumerate(filter_names):
    print(filter)
    q1 = (cq[i,0]+cq[i,1])**2
    q1err = (eq[i,0]**2+eq[i,1]**2)**0.5/(cq[i,0]+cq[i,1])*2*q1
    q2 = cq[i,0]/(2*(cq[i,0]+cq[i,1]))
    q2err = q2*((eq[i,0]/cq[i,0])**2+((eq[i,0]**2+eq[i,1]**2)**0.5/(cq[i,0]+cq[i,1])))**0.5
    # print('q1={}+/-{}. 5-sig = {}'.format(q1,q1err,q1err*5))
    # print('q2={}+/-{}. 5-sig = {}'.format(q2,q2err,q2err*5))
    print("host_ldc_q1_"+str(filter)+","+str(q1)+",1,normal "+str(q1)+" "+str(q1err*3)+",$q_{1; \mathrm{"+str(filter)+"}}$,")
    print("host_ldc_q2_"+str(filter)+","+str(q2)+",1,normal "+str(q2)+" "+str(q2err*3)+",$q_{1; \mathrm{"+str(filter)+"}}$,")

NGTS
c1/c2 values:
0.42515938893603267 0.0006685038402143716
0.13479315041795742 0.0020238795419225413
L
c1/c2 values:
0.5372189986289564 0.0008485716305521009
0.14503210357350832 0.0020313565733018654
....
NGTS
host_ldc_q1_NGTS,0.3135468463289819,1,normal 0.3135468463289819 0.007160991493108053,$q_{1; \mathrm{NGTS}}$,
host_ldc_q2_NGTS,0.37963877208819646,1,normal 0.37963877208819646 0.07028983812613665,$q_{1; \mathrm{NGTS}}$,
L
host_ldc_q1_L,0.4654665664564779,1,normal 0.4654665664564779 0.009011744256743464,$q_{1; \mathrm{L}}$,
host_ldc_q2_L,0.3937106124817453,1,normal 0.3937106124817453 0.06711981838158554,$q_{1; \mathrm{L}}$,
